In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn as s

import matplotlib.pyplot as plt
%matplotlib inline


import math

from sklearn.preprocessing import StandardScaler

# Parte 1

In [ ]:
dfw = pd.read_csv("../input/ccdata/CC GENERAL.csv" ,sep=',',engine='python')
dfw.head()

In [ ]:
dfw.describe()

In [ ]:
dfw.info()

In [ ]:
dfw.isna().sum()


Aca podemos ver que habia datos faltantes en los datos de "CREDIT_LIMIT " y en " MINIMUM_PAYMENTS"

Here we can see that there was missing data in "CREDIT_LIMIT" and in "MINIMUM_PAYMENTS"

In [ ]:
dfh = dfw.copy()
dfh.drop(columns=['CUST_ID'] , axis=1 , inplace=True)

for col in dfh:
    dfh[[col]].hist()

Eliminamos la columna CLUSTID porque no iba a aportar al estudio del caso

In [ ]:
fig = plt.figure(figsize=(12,10))
sns.heatmap(dfw.corr() , annot=True)

# Parte 2

In [ ]:
dfh.isnull().sum()

In [ ]:
dfh.loc[(dfh['MINIMUM_PAYMENTS'].isnull()==True),'MINIMUM_PAYMENTS']=dfh['MINIMUM_PAYMENTS'].mean()
dfh.loc[(dfh['CREDIT_LIMIT'].isnull()==True),'CREDIT_LIMIT']=dfh['CREDIT_LIMIT'].mean()

In [ ]:
dfh.isnull().sum()

In [ ]:
fig = plt.figure(figsize=(20,20))
for col in range(len(dfh.columns)) :
    fig.add_subplot(6,3,col+1)
    sns.boxplot(x=dfh.iloc[ : , col])
plt.show()

Hicimos un Boxplot de los datos para ver si habia outliers y encontramos una gran cantidad de outliers dentro de losdatos 

We did a Boxplot of the data to see if there were outliers and we found a large number of outliers in the data

In [ ]:
for col in dfh:
    dfh[[col]].hist()

### Normalizacion

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(dfh)

Se encontraron muchos outliers por lo que no se podian eliminar ya que se  reduciria notablemente la cantidad de datos, por lo que se decidio  normalizar los datosy asi se evita el problema que genera los outliers y que los datos estan en diferentes escalas 

Many outliers were found so they could not be eliminated since the amount of data would be significantly reduced, so it was decided to normalize the data and avoid the problem that generates outliers.

In [ ]:
dfn = pd.DataFrame(X_scaled) 



In [ ]:
from sklearn import preprocessing

In [ ]:
dfn

In [ ]:
dfn.columns

In [ ]:
dfn.columns = ['BALANCE', 'BALANCE_FREQUENCY', 'PURCHASES',
            'ONEOFF_PURCHASES', 'INSTALLMENTS_PURCHASES', 
            'CASH_ADVANCE', 'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES_FREQUENCY',
            'PURCHASES_INSTALLMENTS_FREQUENCY', 'CASH_ADVANCE_FREQUENCY', 
            'CASH_ADVANCE_TRX', 'PURCHASES_TRX', 'CREDIT_LIMIT', 'PAYMENTS', 
            'MINIMUM_PAYMENTS', 'PRC_FULL_PAYMENT', 'TENURE']

In [ ]:
dfn.describe()

In [ ]:
fig = plt.figure(figsize=(20,20))
for col in range(len(dfn.columns)) :
    fig.add_subplot(6,3,col+1)
    sns.boxplot(x=dfn.iloc[ : , col])
plt.show()

In [ ]:
for col in dfn:
    dfn[[col]].hist()

# Parte 3

### PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca.fit(dfn)
dfc = pca.transform(dfn)

In [ ]:
dfc.shape

Utilizamos PCA para poder reducir la dimensionalidad de los datos y poder graficar los clusters correctamente 

We use PCA to reduce the dimensionality of the data so we could be able to graph the clusters correctly

## Elbow method

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
sse = {}
for k in range(1, 10):
    kmeans = KMeans(n_clusters=k, max_iter=1000).fit(dfn)
    sse[k] = kmeans.inertia_
plt.plot(list(sse.keys()), list(sse.values()))
plt.xlabel("Numero de cluster")
plt.ylabel("SSE")
plt.show()

In [ ]:
wcss= []
range_values = range(1, 20)
for i in range_values:
    kmeans = KMeans(n_clusters=i)
    kmeans.fit(dfc)
    wcss.append(kmeans.inertia_)

In [ ]:

plt.figure(figsize=(15,8)) 
plt.plot(wcss, 'bo-', color='c')
plt.xlabel('Number of clusters Clusters', fontsize=14)
plt.ylabel('WCSS', fontsize=14);

Relizamos el elbow method para poder ver que cantidad de clusters podia ser el mas adecuado con estos datos
Se busca definir la cantidad de clústeres de manera que se minimice la variación total dentro del clúster (conocida como WCSS "quared distance between each point and the centroid in a cluster").
Para determinar el número óptimo de clusters , tenemos que seleccionar el valor de "k" en el "codo", es decir, el punto después del cual la distorsión, inercia comienza a disminuir de forma lineal. Por lo tanto, para los datos dados, concluimos que el número óptimo de clusters para los datos es 3.



We used the elbow method to see how many clusters could be the most appropriate with these data
The aim is to define the number of clusters in such a way as to minimize the total variation within the cluster (known as WCSS "quared distance between each point and the centroid in a cluster").
To determine the optimal number of clusters, we have to select the value of "k" at the "elbow", that is, the point after which the distortion, inertia begins to decrease linearly. Therefore, for the given data, we conclude that the optimal number of clusters for the data is 3.

## K-means

In [ ]:
kmeans =  KMeans(n_clusters=3, max_iter=600, algorithm = 'auto')
kmeans.fit(dfc)

In [ ]:
plt.scatter(dfc[:, 0], dfc[:, 1],
            c= kmeans.labels_.astype(float), edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('Spectral', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

In [ ]:
dfn["cluster"] = kmeans.labels_.astype(float)

In [ ]:
dfn

Colocamos en el dataset a que cluster iba a pertenecer cada persona asi se pueden analizar los datos dividiendolos en clusters y entonces poder ver las caracterisiticas de cada grupo

We put in the dataset which cluster each person was going to belong to, so that the data can be analyzed dividing them into clusters and then we can see the characteristics of each group

In [ ]:
for c in dfn:
    grid= sns.FacetGrid(dfn, col='cluster')
    grid.map(plt.hist, c)    

In [ ]:
dfc

# Parte 4

## DBSCAN

Los algoritmos para clustering basado en densidad identifican regiones de alta densidad que estan rodeadas de areas poco densas en donde cada una de las regiones densas identificadas se corresponde con un cluster.
El concepto principal del algoritmo DBSCAN es localizar regiones de alta densidad que están separadas entre sí por regiones de baja densidad. es un algoritmo de densidad simple que implementa la nocion de densidad por medio de un procedimiento basado en centro.
DBSCAN agrupa los puntos cercanos entre sí en función de una medición de distancia (generalmente distancia euclidiana) y un número mínimo de puntos. También marca como valores atípicos los puntos que se encuentran en regiones de baja densidad.
Es eficiente para grandes volúmenes de datos y Maneja eficientemente la presencia de outliers y ruido. Se ve poco afectado por éstos.

The main concept of the DBSCAN algorithm is to locate high-density regions that are separated from each other by low-density regions. is a simple density algorithm that implements the notion of density by means of a center-based procedure.
DBSCAN groups points that are close to each other based on a distance measurement (usually euclidean distance) and a minimum number of points. It also marks points that are in low-density regions as outliers.
It is efficient for large volumes of data and efficiently handles the presence of outliers and noise.

In [ ]:
pca = PCA(n_components=2)
pca.fit(dfn)
dfd = pca.transform(dfn)

Volvimos a utilizar PCA para poder reducir la dimensionalidad de los datos.

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
db = DBSCAN(eps=0.5, min_samples=10).fit(dfd)
labels = db.labels_

In [ ]:
plt.scatter(dfd[:, 0], dfd[:, 1],
            c= labels.astype(float), edgecolor='none', alpha=0.5,
            cmap=plt.cm.get_cmap('Spectral', 10))
plt.xlabel('component 1')
plt.ylabel('component 2')
plt.colorbar();

In [ ]:
dfn["clusterDBSCAN"] = labels.astype(float)

In [ ]:
dfn

In [ ]:
for c in dfn:
    grid= sns.FacetGrid(dfn, col='clusterDBSCAN')
    grid.map(plt.hist, c)    

In [ ]:
dff = dfn.groupby("cluster")

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
dff.describe(include="all")

In [ ]:
dfh["cluster"] = kmeans.labels_.astype(float)

In [ ]:
dfhh = dfh.groupby("cluster")

In [ ]:
dfh.describe()
for c in dfh:
    grid= sns.FacetGrid(dfh, col='cluster')
    grid.map(plt.hist, c)    

# Parte 5

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
tree_builder =DecisionTreeClassifier(max_depth = 3)
model = tree_builder.fit(dfh[['BALANCE',
'ONEOFF_PURCHASES_FREQUENCY',
'PURCHASES_INSTALLMENTS_FREQUENCY',
'PURCHASES_FREQUENCY',
'MINIMUM_PAYMENTS',
'CREDIT_LIMIT']], dfh["cluster"])

In [ ]:
fig = plt.figure(figsize=(100,100))
tree.plot_tree(model, feature_names=['BALANCE',
'ONEOFF_PURCHASES_FREQUENCY',
'PURCHASES_INSTALLMENTS_FREQUENCY',
'PURCHASES_FREQUENCY',
'MINIMUM_PAYMENTS',
"CREDIT_LIMIT"],
class_names = ['NORMAL', 'NORMAL', 'PREMIUM'],
)

The clusters were divided mainly into two groups. The Premium group who are people with high purchasing power and tend to spend more money and then the Normal group who have a more regular check according to their credit history

A decisiontree was made to be able to indicate through a survey to which group the new clients who do not have a credit history belong.